In [41]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [55]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import re

In [61]:
df = pd.read_csv("/kaggle/input/c-code-from-project-code-net/function_swap_samples.csv")#/kaggle/input/my-data
#df.columns = ["called_function","argument_1","argument_2","type_of_argument","formal_parameter_1","formal_parameter_2","label"]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [62]:
df.isna().sum()

function_name      646
arg1               705
arg2               708
arg_type             3
param1           64505
param2           64498
labels              23
Unnamed: 7       87266
Unnamed: 8       87300
Unnamed: 9       87303
Unnamed: 10      87303
Unnamed: 11      87303
dtype: int64

In [63]:
df.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'],axis=1,inplace=True)
df

,function_name,arg1,arg2,arg_type,param1,param2,labels
0,fopen,file,r,FILE *,__filename,__modes,0
1,fopen,r,file,FILE *,__filename,__modes,1
2,fopen,tempbuf,wt,FILE *,__filename,__modes,0
3,fopen,wt,tempbuf,FILE *,__filename,__modes,1
4,strcmp,ext,extension,int,NaN,NaN,0
...,...,...,...,...,...,...,...
87299,__aeabi_fcmpge,fone,fzero,int,NaN,NaN,1
87300,__aeabi_fcmpge,fone,fzero,int,NaN,NaN,0
87301,__aeabi_fcmpge,fzero,fone,int,NaN,NaN,1
87302,__aeabi_fcmpge,fzero,fNaN,int,NaN,NaN,0


In [64]:
df.isna().sum()

function_name      646
arg1               705
arg2               708
arg_type             3
param1           64505
param2           64498
labels              23
dtype: int64

In [66]:
df['function_name'] = df['function_name'].fillna('[UNK]')
df['arg1']=df['arg1'].fillna('[UNK]')
df['arg2']=df['arg2'].fillna('[UNK]')
df['arg_type']=df['arg_type'].fillna('[UNK]')
df['param1']=df['param1'].fillna('[UNK]')
df['param2']=df['param2'].fillna('[UNK]')
df['labels']=df['labels'].fillna('[UNK]')

In [67]:
df.isna().sum()

function_name    0
arg1             0
arg2             0
arg_type         0
param1           0
param2           0
labels           0
dtype: int64

In [68]:
df['total'] = df['function_name'] + '[SEP]'+ df['arg1'] + '[SEP]'+df['arg2'] + '[SEP]'+ df['arg_type'] + '[SEP]'+df['formal_parameter_2']

KeyError: 'called_function'

In [49]:
df.drop(['called_function','argument_1','argument_2','type_of_argument','formal_parameter_1','formal_parameter_2'],axis=1,inplace=True)

In [50]:
df = df.sample(frac = 1) # shuffling the dataset
df.reset_index(drop=True,inplace=True)

In [51]:
df

,label,total
0,1,glVertex2f[SEP]yc[SEP]int[SEP][UNK][SEP][UNK]
1,1,strcmp[SEP]host[SEP]int[SEP][UNK][SEP][UNK]
2,1,mtdcr[SEP]0x00000000[SEP]int[SEP][UNK][SEP][UNK]
3,1,ACPI_OPTION[SEP]Get table via a physical addre...
4,1,mtdcr[SEP]0x00000056[SEP]int[SEP][UNK][SEP][UNK]
...,...,...
10801,0,glVertex2f[SEP]xc[SEP]int[SEP][UNK][SEP][UNK]
10802,1,strcmp[SEP]workload[SEP]int[SEP][UNK][SEP][UNK]
10803,1,printf[SEP]__func__[SEP]int[SEP][UNK][SEP][UNK]
10804,1,expect[SEP]a[SEP]int[SEP][UNK][SEP][UNK]


In [ ]:
classifier = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
classifier.summary()

In [ ]:
m = len(df)
sample = int(m/10)

In [ ]:
train_df = df[sample:]
test_df = df[:sample]

In [ ]:
def convert_data_to_examples(train, test, code_snippet, label_of_code_snippet): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, 
                                                                           test_df, 
                                                                           'total', 
                                                                           'label')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [ ]:
code_snippet = 'total'
label_of_code_snippet = 'label'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, test_df, code_snippet, label_of_code_snippet)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
# classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
#               metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
# classifier.fit(train_data, epochs=30, validation_data=validation_data,verbose=1,batch_size=64,steps_per_epoch=166)

In [ ]:
my_predict = classifier.predict(test_data)

In [ ]:
param_grid = {
    "per_gpu_batch_size": [16, 32,64],
    "learning_rate": [5e-6, 1e-5, 5e-5],
    "num_epochs": [5, 10, 25,40]
    "steps_per_epoch":[2128,1064,425,266]
}

In [ ]:
from sklearn import model_selection

In [ ]:
model = model_selection.GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=1,
    n_jobs=2,
    cv=5
)

In [ ]:
model.fit(train_data)